In [1]:
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data
import yfinance as yf
%matplotlib inline

# For defining dates
import datetime as dt

# Read from sql
import sqlite3

# Default Values

In [2]:
# Define path to files
PATH = "/Users/jwiegand/Dev/jrwiegand/data/files/"

# Start date defaults
S_YEAR = 2021
S_MONTH = 6
S_DAY = 1
S_DATE_STR = f"{S_YEAR}-{S_MONTH}-{S_DAY}"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

# Start date defaults
E_YEAR = 2024
E_MONTH = 6
E_DAY = 1
E_DATE_STR = f"{E_YEAR}-{E_MONTH}-{E_DAY}"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)


# Manually Download the Screener CSV from Nasdaq
# https://www.nasdaq.com/market-activity/stocks/screener
CSV_DATA_FILE_NAME = "nasdaq_screener_1717356628553.csv"

# Stocks that were not downloaded
stocks_not_downloaded = []

# Import Data from CSV to SQL

In [45]:
def import_csv_to_sqlite(csv_file, db_file, table_name):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)

    # Connect to the SQLite database (creates it if it doesn't exist)
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Create a table with an auto-incrementing ID if it doesn't exist
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        {', '.join([f'`{col.lower().replace(" ", "_").replace("%", "percent")}` TEXT' for col in df.columns])}
    )
    """
    cursor.execute(create_table_query)
    conn.commit()

    # Insert DataFrame records into the database table
    for index, row in df.iterrows():
        insert_query = f"""
        INSERT INTO {table_name} ({', '.join([f'`{col.lower().replace(" ", "_").replace("%", "percent")}`' for col in df.columns])})
        VALUES ({', '.join([f'"{val}"' for val in row])})
        """
        cursor.execute(insert_query)

    conn.commit()
    conn.close()
    print(f"Data from {csv_file} successfully imported into {db_file} in table {table_name}")

# Returns a Named Column Data from a CSV

In [43]:
def get_column_from_csv(file, col_name):
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv(file)
        df[col_name] = df[col_name].astype(str)
        df[col_name] = df[col_name].apply(str)
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df[col_name].astype("string")

# Function that Saves Stock Data to CSV

In [42]:
# Function that gets a dataframe by providing a ticker and starting date
def save_to_csv_from_yahoo(folder, stock_symbol, start, end):
    try:
        print("Get Data for: ", stock_symbol)

        yf.pdr_override()

        # Reads data into a dataframe
        df = web.DataReader(stock_symbol, start, end)["Adj Close"]

        # Save data to a CSV file
        df.to_csv(folder + stock_symbol + ".csv")
    except Exception as ex:
        print(ex)
        stocks_not_downloaded.append(stock_symbol)
        print("Could Not Get Data for :", stock_symbol)

# Test Receiving Stock Tickers

In [46]:
import_csv_to_sqlite(PATH + CSV_DATA_FILE_NAME, PATH + 'stocks.db', 'stocks')

Data from /Users/jwiegand/Dev/jrwiegand/data/files/nasdaq_screener_1717356628553.csv successfully imported into /Users/jwiegand/Dev/jrwiegand/data/files/stocks.db in table stocks


# Get 5 Years of Data for all the Stocks

In [34]:
for ticker in tickers:
  symbol = ticker.replace("/", "-")
  save_to_csv_from_yahoo(PATH + "stocks/", symbol, S_DATE_DATETIME, E_DATE_DATETIME)
print("Finished")
print("Stocks Not Downloaded: ", stocks_not_downloaded)

Get Data for:  A


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Get Data for:  AA



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Get Data for:  AACG
Get Data for:  AACI
Get Data for:  AACIU


...

Get Data for:  MYPS
Get Data for:  MYPSW


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Get Data for:  MYRG
Get Data for:  MYSZ
Get Data for:  MYTE


[*********************100%%**********************]  1 of 1 completed


AttributeError: 'float' object has no attribute 'replace'